In [2]:

import os
from bs4 import BeautifulSoup
import pandas as pd
from lxml import etree


ns = {'tei': 'http://www.tei-c.org/ns/1.0', "xml": "http://www.w3.org/XML/1998/namespace"}



def readlist(pathtofile, filename):
      tree = etree.parse(pathtofile+"/"+filename)
      root = tree.getroot()
      return root

def readfiles(pathtofiles, personlistxml):
    personsFrame = pd.DataFrame()
    personsDict = {}
    personslist = []
    personsref = []
    personstext = []
    personvorkommen = []
    personfiles = []
    personslistvorkommen = []
    personsex =[]
    regestyear = []
    for filename in os.listdir(pathtofiles):
           regesttree = etree.parse(pathtofiles+"/"+filename)
           regestroot = regesttree.getroot()
           personNodes = regestroot.findall('.//*[@type="person"]')
           date = createdate(regestroot.find(".//tei:creation/tei:date", ns))
           for rs in personNodes:
               textnodes = rs.xpath(".//text()")
               text = ' '.join(textnodes[0].replace("\n", "").split())
               if 'ref' in rs.attrib:
                    ref = rs.attrib['ref'].replace("#","")
                    if ref not in personsDict:
                        sublist = [text, 1, [filename], [date]]
                        personsDict[ref] = sublist
                    else:
                        personsDict[ref][1] += 1
                        personsDict[ref][3].append(date)
                        if filename not in personsDict[ref][2]:
                            personsDict[ref][2].append(filename)
               else:
                   personslist.append(text)
                   personslistvorkommen.append(filename)


    for key, item in personsDict.items():
        personsref.append(key)
        personstext.append(item[0])
        personvorkommen.append(item[1])
        filestring = ", ".join(item[2])
        datestring = ", ".join(item[3])
        personfiles.append(filestring)
        personsex.append(getpersonsex(key, personlistxml))
        regestyear.append(datestring)

    for entry in personslist:
        personsref.append('')
        personstext.append(entry)
        personvorkommen.append(1)

    for entry in personslistvorkommen:
        personfiles.append(entry)


    personsFrame['ref'] = personsref
    personsFrame['text'] = personstext
    personsFrame['anzahl'] = personvorkommen
    personsFrame['xml'] = personfiles
    personsFrame['sex'] = personsex
    personsFrame['year'] = regestyear

    personsFrame.to_csv("../data/output/csv/extractedPersons.csv", sep="\t", encoding="utf-8")
    writer = pd.ExcelWriter('../data/output/xlsx/extractedPersons.xlsx')
    personsFrame.to_excel(writer, 'ExtractedPersons', index=False)
    writer.save()

def getpersonsex(ref, personlist):
    ref = ref.replace("#", "")
    person = personlist.find(".//tei:person[@xml:id='"+ref+"']", ns)
    if person is not None:
        if "sex" in person.attrib:
            sex = person.attrib['sex']
        else: sex = "-"
        return sex
    else:
        return "-"


def createdate(node):
    if 'when' in node.attrib:
        when = node.attrib['when']
        year = node.attrib['when'][:4]
        return year


personlistxml = readlist("../../data/lists", "personList.xml")    
readfiles("../../regests_QGW_PR_Stadt_ab_1400/done", personlistxml)

